In [1]:
from netCDF4 import Dataset, num2date
from pathlib import Path

In [ ]:
# install era5cli
!pip install era5cli

In [ ]:
# path to forcing data
forcing_path  = "/data/shared/EcoExtreML/STEMMUS_SCOPEv1.0.0/input/Plumber2_data/"

## Test run using --dryrun

In [4]:
forcing_name = "AR-SLu_2010-2010_FLUXNET2015_Met.nc"
# read data
filename = Path(forcing_path, forcing_name) # care for windows/unix paths
nc_fid = Dataset(filename, mode='r')

nctime = nc_fid.variables['time']
ncdate = num2date(nctime[:],units= nctime.units, calendar=nctime.calendar)
year = [date.year for date in ncdate]

startyear = year[0]
endyear = year[-1]

lat = float(nc_fid.variables['latitude'][:].flatten()[0])
lon = float(nc_fid.variables['longitude'][:].flatten()[0])

In [5]:
print(startyear, endyear, lat, lon)

2010 2010 -33.46480178833008 -66.45980072021484


In [10]:
lat_max = lat + 0.001
lat_min = lat - 0.001
lon_max = lon + 0.001
lon_min = lon - 0.001
print(lat_max, lon_min, lat_min, lon_max)

-33.46380178833008 -66.46080072021485 -33.465801788330076 -66.45880072021484


In [14]:
!era5cli hourly --variables soil_temperature_level_1 volumetric_soil_water_layer_1 --startyear {startyear} --endyear {endyear} --land --area {lat_max} {lon_min} {lat_min} {lon_max} --outputprefix AR-SLu --dryrun --merge

NB: coordinates [-33.46380178833008, -66.46080072021485, -33.465801788330076, -66.45880072021484] rounded down to two decimals.

NB: coordinates [-33.46380178833008, -66.46080072021485, -33.465801788330076, -66.45880072021484] rounded down to two decimals.

reanalysis-era5-land {'variable': 'soil_temperature_level_1', 'year': [2010], 'month': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'], 'time': ['00:00', '01:00', '02:00', '03:00', '04:00', '05:00', '06:00', '07:00', '08:00', '09:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00', '16:00', '17:00', '18:00', '19:00', '20:00', '21:00', '22:00', '23:00'], 'format': 'netcdf', 'area': [-33.46, -66.46, -33.47, -66.46], 'day': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']} AR-SLu-land_soil_temperature_level_1_2010_hourly_66W-66W_33S-33S.nc
reanalysis-era5-land {'variabl

## Download data for 170

In [ ]:
forcing_filenames_list = [file.name for file in Path(forcing_path).iterdir()]
for forcing_name in forcing_filenames_list:
    # read data
    filename = Path(forcing_path, forcing_name) # care for windows/unix paths
    nc_fid = Dataset(filename, mode='r')

    nctime = nc_fid.variables['time']
    ncdate = num2date(nctime[:],units= nctime.units, calendar=nctime.calendar)
    year = [date.year for date in ncdate]

    startyear = year[0]
    endyear = year[-1]

    lat = float(nc_fid.variables['latitude'][:].flatten()[0])
    lon = float(nc_fid.variables['longitude'][:].flatten()[0])
    
    # create area
    lat_max = lat + 0.001
    lat_min = lat - 0.001
    lon_max = lon + 0.001
    lon_min = lon - 0.001
    
    # Download data
    station_name = ncfile.split('_')[0]
    !era5cli hourly --variables soil_temperature_level_1 volumetric_soil_water_layer_1 --startyear {startyear} --endyear {endyear} --land --area {lat_max} {lon_min} {lat_min} {lon_max} --outputprefix {station_name} --merge
    